## 小麦の形状をCSVの表にする。

komugi_inのフォルダーを作成し、そこに画像をアップロードしこの分の上部にある▶のボタンを押す。

面積のわかっているスケールを用意する。同じ面積のものを赤色と、青色で用意する。

scalearea_mm2はその面積を平方ミリメートルで入力。

データの出力先は、outdirで指定したディレクトリの中data.csvが統計情報

CPUとGPUどちらでも実行可能。

In [9]:
test_dir = "komugi_in"
outdir = "komugi_out"
scalearea_mm2 = 25.289

In [ ]:
!pip install ultralytics

In [11]:
from ultralytics import YOLO

In [12]:
!git clone https://github.com/sou-06/Wheat_Seed_Phenotype

fatal: destination path 'Wheat_Seed_Phenotype' already exists and is not an empty directory.


In [13]:
import cv2
import numpy as np
import os
import shutil
def scaleArea(img,imgpath):
    # HSV色空間に変換（色抽出はBGRよりHSVの方がやりやすい）
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 青色の範囲
    lower_blue = np.array([100, 50, 50])
    upper_blue = np.array([130, 255, 255])

    # 赤色の範囲（赤はHSVで2つに分かれるので両方作る）
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([179, 255, 255])

    # マスク作成
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = mask_red1 | mask_red2

    kernel = np.ones((5,5), np.uint8)
    mask_blue = cv2.morphologyEx(mask_blue, cv2.MORPH_OPEN, kernel)
    mask_blue = cv2.morphologyEx(mask_blue, cv2.MORPH_CLOSE, kernel)
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, kernel)
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, kernel)

    os.makedirs(outdir + "/scale", exist_ok=True)
    cv2.imwrite(outdir + "/scale/blue" + os.path.basename(imgpath), mask_blue)
    cv2.imwrite(outdir + "/scale/red" + os.path.basename(imgpath), mask_red)
    def get_largest_area(mask):
        # 輪郭を抽出
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            return 0
        # 面積の最大値を返す
        return max(cv2.contourArea(c) for c in contours)

    # 青と赤の最大面積を算出
    blue_area = get_largest_area(mask_blue)
    red_area = get_largest_area(mask_red)
    ave = (blue_area + red_area) / 2
    if ave < 0.01 or abs(blue_area - red_area) / ave > 0.1:
        return 0.001
    else:
        return ave

In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import pandas as pd
import os
import math
import shutil
model1 = YOLO("Wheat_Seed_Phenotype/wheat_model.pt")
model2 = YOLO("Wheat_Seed_Phenotype/wheat_model2.pt")
def predict_seed(inputpath,save_dir):

    img_path = inputpath
    img = cv2.imread(img_path)

    # 出力保存フォルダ
    if os.path.exists("seed_crops") and os.path.isdir("seed_crops"):
        shutil.rmtree("seed_crops")
    os.makedirs("seed_crops", exist_ok=True)

    height, width, channels = img.shape[:3]
    # 推論
    results1 = model1.predict(inputpath, save=True, conf=0.7, imgsz = 1408, half = True)
    img = np.copy(results1[0].orig_img)
    b_mask = np.zeros(img.shape[:2], np.uint8)
    overlay = img.copy()
    # img_rgb にコピー
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    H, W = img.shape[:2]

    all_data = []
    # 各マスクを重ねる
    j = 1
    scale = scaleArea(img,inputpath)

    for i, box in enumerate(results1[0].boxes.xyxy.cpu().numpy()):
        x1, y1, x2, y2 = map(int, box)
        lx =  max(1, x1 - 30)
        ly = max(1, y1 - 30)
        ry = min(height - 2, y2 + 30)
        rx = min(width - 2, x2 + 30)
        # 種子の切り出し
        crop = img[ly:ry,  lx:rx]
        out_path = "seed_crops/" + os.path.splitext(os.path.basename(inputpath))[0] + f"{i:03d}.jpg"
        cv2.imwrite(out_path, crop)
        ch, cw = crop.shape[:2]
        # 種子ごとに再推論（高解像度セグメント）
        results = model2.predict(crop, imgsz=1024, conf=0.8)

        if results[0].masks is None:
            continue
        for m in results[0].masks.xy:
            contour = m.astype(np.int32).reshape(-1, 1, 2)
            mask = np.zeros((ch , cw), dtype=np.uint8)

            kernel = np.ones((3,3), np.uint8)
            maskFull = np.zeros((H, W), dtype=np.uint8)

            # ② 輪郭を塗りつぶし
            cv2.fillPoly(mask, [contour], 255)
            #cv2.imwrite(save_dir + f"{i:03d}.bmp", mask)
            # ③ モルフォロジー処理
            kernel = np.ones((8,8), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            maskFull[ly - 1:ry - 1, lx - 1 :rx - 1] = mask
            contours, _ = cv2.findContours(maskFull, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            for cnt in contours:
                if len(cnt) < 120:
                    continue

                flag = True
                for p in cnt:
                    if p[0][0] <= lx + 1 or p[0][1] <= ly + 1 or \
                       p[0][0] >= rx - 2 or p[0][1] >= ry - 2:
                        flag = False
                        break
                if flag == False: continue

                epsilon = 0.001 * cv2.arcLength(cnt, True)
                cnt_smooth = cv2.approxPolyDP(cnt, epsilon, True)
                cv2.drawContours(overlay, [cnt_smooth], -1, (0, 255, 0), 2)
                # 楕円フィッティング
                ellipse = cv2.fitEllipse(cnt)

                # 楕円面積
                (x_center, y_center), (minor_axis,major_axis), angle = ellipse
                area_ellipse = np.pi * (major_axis / 2) * (minor_axis / 2)
                px_per_mm = math.sqrt(scale) / math.sqrt(scalearea_mm2)
                if abs(cv2.contourArea(cnt) - area_ellipse) /  cv2.contourArea(cnt) > 0.05:
                    continue

                cv2.putText(overlay, str(j), (int(x_center) - 7, int(y_center) - 7), cv2.FONT_HERSHEY_DUPLEX, 0.8, color = (0, 0, 255))
                cv2.ellipse(overlay, ellipse, color=(0, 0, 255), thickness = 2)  # 赤色で描画
                all_data.append({
                    "File_Name": os.path.basename(inputpath).split(".")[0],
                    "Object_ID": j,
                    "Center_X": x_center,
                    "Center_Y": y_center,
                    "Major_Axis_px": major_axis,
                    "Minor_Axis_px": minor_axis,
                    "Angle": angle,
                    "Ellipse_Area_px": area_ellipse,
                    "scale_px" : scale,
                    "px/mm" : px_per_mm,
                    "length_mm" : major_axis / px_per_mm,
                    "width_mm": minor_axis / px_per_mm,
                    "Ellipticity" : minor_axis / major_axis,
                    "Area_mm2" : area_ellipse / scale * scalearea_mm2
                })
                j += 1

    # 6. 結果をCSV出力
    if save_dir[-1] != "/":
        save_dir += "/"
    df = pd.DataFrame(all_data)
    csv_path = inputpath.replace(".JPG", ".csv")
    df.to_csv( save_dir + "data.csv", index=False,header=False, mode='a')
    print(f"Results saved to {csv_path}")
    #print(df)

    # 7. 可視化結果を保存
    os.makedirs(save_dir, exist_ok=True)
    results[0].save(save_dir + "res.png")
    print(f"Detection results saved in {save_dir}/")

    # 半透明にする場合
    alpha = 0.5
    #overlay[b_mask>0] = [255, 0, 0]  # RGB赤
    img_display = cv2.addWeighted(overlay, alpha, img, 1-alpha, 0)
    cv2.imwrite(save_dir + os.path.basename(inputpath), img_display)
    img_display2 = cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB)


In [ ]:
import cv2
import os
import glob

os.makedirs(outdir, exist_ok=True)

f = open(outdir + "/data.csv", mode = "w")
d = [
        "File_Name",
        "Object_ID",
        "Center_X",
        "Center_Y",
        "Major_Axis_px",
        "Minor_Axis_px",
        "Angle",
        "Ellipse_Area_px",
        "scale_px",
        "px/mm",
        "length_mm",
        "width_mm",
        "Ellipticity",
        "Area_mm2"
    ]
f.write(",".join(d))
f.write("\n")
f.close()
for jpeg_path in glob.glob(os.path.join(test_dir, "*.JPG")):
    predict_seed(jpeg_path, outdir)
